In [ ]:
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import pandas as pd
import matplotlib.pyplot as plt
from yandexgptlite import YandexGPTLite
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import tempfile
import openpyxl
from datetime import datetime

# Авторизация в Google Sheets
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
gc = gspread.authorize(credentials)
sh = gc.open_by_key('######################################')


data = pd.DataFrame(columns=['Date'])

#  YandexGpt
account = YandexGPTLite('#######################', '###################################')
# создаем объект бота и передаем токен авторизации
bot = telegram.Bot(token='00000000000:##############################')

# функция-обработчик команды /start
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Привет! Я ваш личный тренер. Я могу записывать ваши успехи и составлять тренировки. Напишите /help чтобы узнать о всех возможностях")
def help(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="Список доступных команд: 1) /generate_text - возращает Бориса 2)  /summarization - краткий пересказ")

# функция-обработчик сообщений от пользователя
def generate_text(update, context):
    # Получаем текст сообщения от пользователя
    user_text = update.message.text
    # Генерируем текст с помощью Yandex GPT
    generated_text = account.create_completion( user_text, '0.5', model= 'lite', system_prompt = 'Отвечай как тренер по фитнесу с 20 летним опытом')
    # Отправляем сгенерированный текст пользователю в Telegram
    context.bot.send_message(chat_id=update.effective_chat.id, text=generated_text)


def create_user_sheet(user_id):
    try:
        user_sheet = sh.add_worksheet(title=str(user_id), rows="100", cols="4")
        user_sheet.update([data.columns.tolist()])
    except gspread.exceptions.APIError:
        user_sheet = sh.worksheet(str(user_id))
    return user_sheet

def add_value(update, context):
    user_id = update.effective_user.id
    user_sheet = create_user_sheet(user_id)
    values_list = user_sheet.get_all_values()[1:]
    data = pd.DataFrame(values_list, columns = user_sheet.get_all_values()[0])
    try:
        exercise, value = update.message.text.split()[1:]
        # exercise = exercise.str.lower()
        value = int(value)
        user_sheet = create_user_sheet(user_id)
        if exercise in data.columns:
            data.loc[len(data), exercise] = value
            data.loc[len(data) - 1, 'Date'] = datetime.now().date()
            data['Date'] = data['Date'].astype(str)
            data[exercise] = data[exercise].astype(str)
            last_row = data.fillna(0).iloc[-1].values.tolist()
            user_sheet.append_row(last_row)
            update.message.reply_text(f'Значение {value} для упражнения "{exercise}" успешно добавлено в таблицу.')
        else:
            update.message.reply_text(f'Упражнение "{exercise}" не существует в таблице.')
    except ValueError:
        update.message.reply_text('Пожалуйста, укажите упражнение и значение для добавления в таблицу.')

def add_column(update, context):
    user_id = update.effective_user.id
    user_sheet = create_user_sheet(user_id)
    values_list = user_sheet.get_all_values()[1:]
    data = pd.DataFrame(values_list, columns=user_sheet.get_all_values()[0])
    try:
        column_name = update.message.text.split()[1]
        if column_name not in data.columns:
            user_sheet.add_cols(1)
            data[column_name] = ''
            if 'Date' not in data.columns:
                user_sheet.add_cols(1)
                data.insert(0, 'Date', '')
            user_sheet.update([data.columns.tolist()])
            update.message.reply_text(f'Столбец "{column_name}" успешно добавлен в таблицу.')
        else:
            update.message.reply_text(f'Столбец "{column_name}" уже существует в таблице.')
    except IndexError:
        update.message.reply_text('Пожалуйста, укажите название столбца для добавления в таблицу.')




def plot_graph(update, context):
    exercise = update.message.text.split()[1]
    user_id = update.effective_user.id
    user_sheet = create_user_sheet(user_id)
    values_list = user_sheet.get_all_values()[1:]
    data = pd.DataFrame(values_list, columns = user_sheet.get_all_values()[0])
    # data.columns = data[0]
    # data = pd.DataFrame(values_list, columns=udata.columns) #['Жим', 'Трицепс', 'Бицепс', 'Ноги'] data.columns  ['Жим', 'Трицепс', 'Бицепс', 'Ноги', 'Тест']
    if not data.empty:
        try:
            # exercise = update.message.text.split()[1]
            last_10_data = data[exercise].tail(10).replace('',0).astype(int).replace(0,np.nan).dropna().astype(int)#.replace(0,np.nan).dropna().astype(int)
            if not last_10_data.empty:
                plt.figure()
                plt.plot(last_10_data.index, last_10_data)
                plt.ylabel('Вес')
                plt.xlabel('Номер тренировки')
                plt.title(f'График упражнения "{exercise}"')
                plt.grid(True)
                graph_file = 'graph.png'
                plt.savefig(graph_file)
                update.message.reply_photo(photo=open(graph_file, 'rb'))
            else:
                update.message.reply_text('В таблице недостаточно данных для построения графика.')
        except (ValueError, IndexError):
            update.message.reply_text('Пожалуйста, укажите упражнение для построения графика.')
    else:
        update.message.reply_text('В таблице нет данных для построения графика.')

def show_table(update, context):
    user_id = update.effective_user.id
    user_sheet = create_user_sheet(user_id)
    values_list = user_sheet.get_all_values()
    if len(values_list) > 1:
        data = pd.DataFrame(values_list[1:], columns=values_list[0])
        
        # Сохраняем DataFrame в файл Excel
        excel_file_path = tempfile.NamedTemporaryFile(suffix='.xlsx').name
        data.to_excel(excel_file_path, index=False)

        # Отправляем файл Excel пользователю
        update.message.reply_document(open(excel_file_path, 'rb'), filename='table.xlsx')
        # update.message.reply_text(data.to_string())
    else:
        update.message.reply_text('В таблице нет данных.')

def clear_data(update, context):
    user_id = update.effective_user.id
    user_sheet = create_user_sheet(user_id)
    user_sheet.clear()
    update.message.reply_text('Таблица очищена.')
    

# создаем объект Updater для обработки обновлений от телеграмма
updater = Updater(bot.token, use_context=True)

# создаем объект диспетчера для регистрации обработчиков команд и сообщений
dispatcher = updater.dispatcher

# регистрируем обработчик команды /start
# Добавление обработчика команды /addcolumn
dispatcher.add_handler(CommandHandler("add_column", add_column))
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(CommandHandler("help", help))
dispatcher.add_handler(CommandHandler("generate_text", generate_text))
dispatcher.add_handler(CommandHandler('add_value', add_value))
dispatcher.add_handler(CommandHandler('plot_graph', plot_graph))
dispatcher.add_handler(CommandHandler('show_table', show_table))
dispatcher.add_handler(CommandHandler("clear_data", clear_data))

# Создаем экземпляр класса telegram.ext.Updater и добавляем обработчики команд и сообщений
dispatcher = updater.dispatcher
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, generate_text))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, add_value))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, plot_graph))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, show_table))

# Запускаем бота
updater.start_polling()
updater.idle()